### [目的]<br>db.sqlite3にレコードされた来店データを取得し, pandasで整形する.

In [1]:
from pathlib import Path
Path.cwd()

PosixPath('/root')

In [3]:
from pathlib import Path
import os, sys, django

# コンテナ内 Django プロジェクトのルート
project_root = Path("/app")

print("Using project root:", project_root)

# PYTHONPATH に追加
sys.path.insert(0, str(project_root))

# settings モジュールを指定
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')

# Django 初期化
django.setup()

print("Django version:", django.get_version())

Using project root: /app
Django version: 4.2.20


In [39]:
from pathlib import Path
import os, sys, django
import pandas as pd
import sqlite3

# --- Django 初期化 ---
project_root = Path("/app")
sys.path.insert(0, str(project_root))
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')
django.setup()

# --- SQLite 接続 via Django settings ---
from django.conf import settings
db_path = settings.DATABASES['default']['NAME']
conn = sqlite3.connect(db_path)

# --- DataFrame にロード ---
query = """
SELECT id, date, table_number, people_count,plan_name, start_time, end_time, out_time, extensions, invoiceChecked, paymentChecked
FROM TimeManagement_timemanagementrecord
WHERE table_number IS NOT NULL
"""
df = pd.read_sql_query(query, conn)

# フォーマット調整
df['start_time'] = df['start_time'].astype(str).str.zfill(4).str.replace(r'(\d{2})(\d{2})', r'\1:\2', regex=True)
df['end_time']   = df['end_time'].astype(str).str.zfill(4).str.replace(r'(\d{2})(\d{2})', r'\1:\2', regex=True)


nulls = df["people_count"].isnull().sum()
print(f"poeple_count欠損値合計数 : {nulls}")
df

poeple_count欠損値合計数 : 64


,id,date,table_number,people_count,plan_name,start_time,end_time,out_time,extensions,invoiceChecked,paymentChecked
0,1,2025-02-25,5,2.0,スーパー,15:20,00:00,,【アウト】17:20,1,1
1,2,2025-02-25,"5, 6",11.0,食べ飲み120m,14:00,00:00,,【アウト】16:00,1,1
2,3,2025-02-25,1,3.0,レモン,18:15,20:58,21:30,12,1,1
3,4,2025-02-25,,NaN,,00:00,00:00,,,0,0
4,5,2025-02-25,9,2.0,スーパー,18:55,20:20,,【アウト】20:55,1,1
...,...,...,...,...,...,...,...,...,...,...,...
127,128,2025-03-27,16,2.0,スーパー,18:45,00:00,,【アウト】20:45,1,1
128,129,2025-03-27,2,2.0,レモン,19:23,20:19,20:50,0,1,1
129,130,2025-03-27,14,6.0,食べ飲み90m,19:25,00:00,,【アウト】20:55,1,0
130,131,2025-03-27,"5, 6",12.0,食べ飲み90m,21:00,00:00,,【アウト】21:30,0,0


In [ ]:
# people_count欠損値除去 → float型からint型へ変換
df = df.dropna(subset=['people_count']).copy()  # copy()を付けてビューではなくコピーにする
df.loc[:, 'people_count'] = df['people_count'].astype(int)

# 新たなカラムseat_timeを作成
df['seat_time'] = None

# すでにpeople_countをintにしているので再度変換する必要はないですが、念のため
df["people_count"] = df["people_count"].astype(int)

# ここを変更：レモンの場合は、out_timeが空の場合は"23:59"、そうでなければout_timeをそのまま採用
mask_lemon = df['plan_name'] == 'レモン'
df.loc[mask_lemon, 'seat_time'] = np.where(
    df.loc[mask_lemon, 'out_time'].isna() | (df.loc[mask_lemon, 'out_time'] == ""),
    "23:59",
    df.loc[mask_lemon, 'out_time']
)

# その他のプランの場合：extensionsから値を取得（"【アウト】"を除去）
mask = df['plan_name'].isin(['食べ飲み90m', '食べ飲み120m', 'スーパー'])
df.loc[mask, 'seat_time'] = df.loc[mask, 'extensions'].str.replace('【アウト】', '', regex=False)

# statusカラムの作成(invoiceCheckedとpaymentCheckedの合計値)
df['status'] = df['invoiceChecked'].astype(int) + df['paymentChecked'].astype(int)

# 最終的にseat_timeが欠損している場合は "23:59" で補完
df['seat_time'] = df['seat_time'].fillna("23:59")

df.tail(10)

,id,date,table_number,people_count,plan_name,start_time,end_time,out_time,extensions,invoiceChecked,paymentChecked,seat_time,status
108,109,2025-03-26,A1,2,単品,21:18,23:30,,,1,1,23:59,2
109,110,2025-03-27,5,3,レモン,18:03,00:00,,,0,0,23:59,0
110,111,2025-03-27,13,2,スーパー,19:00,00:00,,【アウト】21:00,1,1,21:00,2
111,112,2025-03-27,9,2,スーパー,18:10,00:00,,【アウト】20:10,1,1,20:10,2
125,126,2025-03-27,A1,1,レモン,18:25,19:23,19:55,0,1,1,19:55,2
126,127,2025-03-27,11,3,レモン,18:29,19:23,19:55,0,1,1,19:55,2
127,128,2025-03-27,16,2,スーパー,18:45,00:00,,【アウト】20:45,1,0,20:45,1
128,129,2025-03-27,2,2,レモン,19:23,20:19,20:50,0,1,1,20:50,2
129,130,2025-03-27,14,6,食べ飲み90m,19:25,00:00,,【アウト】20:55,1,0,20:55,1
130,131,2025-03-27,A6,2,食べ飲み90m,19:50,00:00,,,0,0,,0


In [40]:
df = df.dropna(subset=['people_count']).copy()
df['people_count'] = df['people_count'].astype(int)
for col in ['start_time','end_time','out_time']:
    df[col] = df[col].astype(str).str.zfill(4).str.replace(r'(\d{2})(\d{2})', r'\1:\2', regex=True)

df['seat_time'] = None

# レモンプランの場合：out_time が "00:00" なら "23:59"、そうでなければ out_time をそのまま採用
mask_lemon = df['plan_name'] == 'レモン'
df.loc[mask_lemon, 'seat_time'] = df.loc[mask_lemon, 'out_time'].apply(
    lambda t: "23:59" if t == "00:00" else t
)

# その他のプランの場合：extensions から値を取得（"【アウト】"を除去）
mask = df['plan_name'].isin(['食べ飲み90m','食べ飲み120m','スーパー'])
df.loc[mask, 'seat_time'] = df.loc[mask, 'extensions'].str.replace('【アウト】','', regex=False)

# statusカラムの作成
df['status'] = df['invoiceChecked'].astype(int) + df['paymentChecked'].astype(int)

# テスト段階のため、seat_timeに欠損値のある行は削除
df = df.dropna(subset=['seat_time']).copy()

df.tail(10)

,id,date,table_number,people_count,plan_name,start_time,end_time,out_time,extensions,invoiceChecked,paymentChecked,seat_time,status
107,108,2025-03-26,14,2,レモン,21:24,22:24,22:55,0,1,1,22:55,2
109,110,2025-03-27,5,3,レモン,18:03,20:03,20:35,6,1,1,20:35,2
110,111,2025-03-27,13,2,スーパー,19:00,00:00,00:00,【アウト】21:00,1,1,21:00,2
111,112,2025-03-27,9,2,スーパー,18:10,00:00,00:00,【アウト】20:10,1,1,20:10,2
125,126,2025-03-27,A1,1,レモン,18:25,19:23,19:55,0,1,1,19:55,2
126,127,2025-03-27,11,3,レモン,18:29,19:23,19:55,0,1,1,19:55,2
127,128,2025-03-27,16,2,スーパー,18:45,00:00,00:00,【アウト】20:45,1,1,20:45,2
128,129,2025-03-27,2,2,レモン,19:23,20:19,20:50,0,1,1,20:50,2
129,130,2025-03-27,14,6,食べ飲み90m,19:25,00:00,00:00,【アウト】20:55,1,0,20:55,1
130,131,2025-03-27,"5, 6",12,食べ飲み90m,21:00,00:00,00:00,【アウト】21:30,0,0,21:30,0
